In [1]:
import sigpde.torch as sig
import torch
import math
from samplers import *
device = torch.device('cuda:0')

In [4]:
kernel = sig.RobustSigPDE(sig.kernels.LinearKernel(), 1)

In [2]:
x = generate(8, 512, 50, device=device)
y = generate(4, 512, 50, device=device)
x = NonLinearSDE.sample(8, theta=1.5, thin=1)
z, w = sample(8)
z = add_time(z)
w = add_time(w)
y = add_time(y)

In [3]:
x, y = NonLinearSDE.sample(n=8, theta=1.5, thin=1)
x = x
x = add_time(x)
y = add_time(y)

In [8]:
kernel.gram(x)

tensor([[1.5687, 1.3394, 1.4185, 1.3611, 1.4439, 1.5328, 1.3100, 1.3527],
        [1.3394, 1.5784, 1.1883, 1.3075, 1.2312, 1.2962, 1.5830, 1.4677],
        [1.4185, 1.1883, 1.4928, 1.4543, 1.4818, 1.4651, 1.1776, 1.3452],
        [1.3611, 1.3075, 1.4543, 1.5422, 1.4484, 1.4230, 1.3165, 1.4723],
        [1.4439, 1.2312, 1.4818, 1.4484, 1.4781, 1.4778, 1.2187, 1.3628],
        [1.5328, 1.2962, 1.4651, 1.4230, 1.4778, 1.5365, 1.2747, 1.3727],
        [1.3100, 1.5830, 1.1776, 1.3165, 1.2187, 1.2747, 1.5927, 1.4792],
        [1.3527, 1.4677, 1.3452, 1.4723, 1.3628, 1.3727, 1.4792, 1.5175]],
       device='cuda:0', dtype=torch.float64)